# LLM API – End-to-End Examples (Single Notebook)

This notebook shows a minimal client and step-by-step examples for:

1. Create a new account
2. Login
3. Change models (admin privilege example)
4. Start a new chat and get a response
5. Continue a chat
6. See chat history
7. Websearch with agentic tool selection
8. Agentic math calculation (LLM decides to use math tool)
9. Sequential reasoning with ReAct agent (step-by-step thinking)
10. Plan-and-Execute agent (parallel tool usage)
11. Auto agent selection (smart router picks best agent)
12. Complex JSON data analysis
13. Real Data Analysis - Warpage Statistics (using 20251013_stats.json)
14. Python Code Generation - Simple Calculation
15. Python Code Generation - Data Analysis
16. Python Code Generation - Mathematical Computation
17. Python Code Generation - String Processing
18. Python Code Generation - Excel File Analysis (Real File)

Set your API base URL below if different from the default.

In [ ]:
import sys
# !{sys.executable} -m pip install httpx
# ! pip install pip-system-certs

In [ ]:
# API_BASE_URL = "http://10.252.38.241:10007"
API_BASE_URL = 'http://localhost:10007'
print("Using:", API_BASE_URL)

In [ ]:
import httpx
import json
from pathlib import Path
from typing import Iterator

class LLMApiClient:
    def __init__(self, base_url: str, timeout: float = 3600.0):
        self.base_url = base_url.rstrip("/")
        self.token = None
        # Create timeout config: 10s for connect, custom timeout for read/write/pool
        self.timeout = httpx.Timeout(50.0, read=timeout, write=timeout, pool=timeout)

    def _headers(self):
        # Don't set Content-Type - httpx auto-sets for multipart
        h = {}
        if self.token:
            h["Authorization"] = f"Bearer {self.token}"
        return h

    def signup(self, username: str, password: str, role: str = "guest"):
        r = httpx.post(f"{self.base_url}/api/auth/signup", json={
            "username": username, "password": password, "role": role
        }, timeout=10.0)
        r.raise_for_status()
        return r.json()

    def login(self, username: str, password: str):
        r = httpx.post(f"{self.base_url}/api/auth/login", json={
            "username": username, "password": password
        }, timeout=10.0)
        r.raise_for_status()
        data = r.json()
        self.token = data["access_token"]
        return data

    def list_models(self):
        # JSON endpoints still use Content-Type header
        headers = {"Authorization": f"Bearer {self.token}"} if self.token else {}
        r = httpx.get(f"{self.base_url}/v1/models", headers=headers, timeout=10.0)
        r.raise_for_status()
        return r.json()

    def change_model(self, model: str):
        headers = {"Authorization": f"Bearer {self.token}", "Content-Type": "application/json"} if self.token else {"Content-Type": "application/json"}
        r = httpx.post(f"{self.base_url}/api/admin/model", json={"model": model}, headers=headers, timeout=10.0)
        r.raise_for_status()
        return r.json()

    def chat_new(self, model: str, user_message: str, agent_type: str = "auto", files: list = None):
        messages = [{"role": "user", "content": user_message}]
        
        # Prepare form data
        data = {
            "model": model,
            "messages": json.dumps(messages),
            "agent_type": agent_type
        }
        
        # Prepare files for upload
        files_to_upload = []
        if files:
            for file_path in files:
                f = open(file_path, "rb")
                files_to_upload.append(("files", (Path(file_path).name, f)))
        
        try:
            r = httpx.post(
                f"{self.base_url}/v1/chat/completions",
                data=data,
                files=files_to_upload if files_to_upload else None,
                headers=self._headers(),
                timeout=self.timeout
            )
            r.raise_for_status()
            result = r.json()
            return result["choices"][0]["message"]["content"], result["x_session_id"]
        
        finally:
            # Close file handles
            for _, (_, f) in files_to_upload:
                f.close()

    def chat_new_streaming(self, model: str, user_message: str, agent_type: str = "auto", files: list = None) -> Iterator[str]:
        """
        Start new chat with streaming response (Server-Sent Events)
        
        Args:
            model: Model name
            user_message: User message
            agent_type: Agent type (auto, react, plan_execute) - Note: streaming only works for simple chat
            files: Optional list of file paths to attach
        
        Yields:
            Response tokens as they're generated
            
        Returns:
            Iterator[str]: Yields tokens, then yields session_id as final value with prefix "SESSION_ID:"
        """
        messages = [{"role": "user", "content": user_message}]
        
        # Prepare form data
        data = {
            "model": model,
            "messages": json.dumps(messages),
            "agent_type": agent_type,
            "stream": "true"  # Enable streaming
        }
        
        # Prepare files for upload
        files_to_upload = []
        if files:
            for file_path in files:
                f = open(file_path, "rb")
                files_to_upload.append(("files", (Path(file_path).name, f)))
        
        try:
            with httpx.stream(
                "POST",
                f"{self.base_url}/v1/chat/completions",
                data=data,
                files=files_to_upload if files_to_upload else None,
                headers=self._headers(),
                timeout=self.timeout
            ) as response:
                response.raise_for_status()
                
                session_id = None
                for line in response.iter_lines():
                    if line.startswith("data: "):
                        data_str = line[6:]  # Remove "data: " prefix
                        
                        if data_str == "[DONE]":
                            # Stream complete
                            break
                        
                        try:
                            chunk = json.loads(data_str)
                            
                            # Check for errors
                            if "error" in chunk:
                                raise Exception(f"Streaming error: {chunk['error']['message']}")
                            
                            # Extract session_id from final chunk
                            if "x_session_id" in chunk:
                                session_id = chunk["x_session_id"]
                            
                            # Yield content delta
                            if "choices" in chunk and len(chunk["choices"]) > 0:
                                delta = chunk["choices"][0].get("delta", {})
                                if "content" in delta:
                                    yield delta["content"]
                        
                        except json.JSONDecodeError:
                            # Skip malformed JSON
                            continue
                
                # Yield session_id at the end with special prefix
                if session_id:
                    yield f"SESSION_ID:{session_id}"
        
        finally:
            # Close file handles
            for _, (_, f) in files_to_upload:
                f.close()

    def chat_continue(self, model: str, session_id: str, user_message: str, agent_type: str = "auto", files: list = None):
        """
        Continue existing chat with optional file attachments (multipart/form-data)
        
        Args:
            model: Model name
            session_id: Session ID to continue
            user_message: User message
            agent_type: Agent type (auto, react, plan_execute)
            files: Optional list of file paths to attach
        
        Returns:
            Tuple of (response_text, session_id)
        """
        messages = [{"role": "user", "content": user_message}]
        
        data = {
            "model": model,
            "messages": json.dumps(messages),
            "session_id": session_id,
            "agent_type": agent_type
        }
        
        files_to_upload = []
        if files:
            for file_path in files:
                f = open(file_path, "rb")
                files_to_upload.append(("files", (Path(file_path).name, f)))
        
        try:
            r = httpx.post(
                f"{self.base_url}/v1/chat/completions",
                data=data,
                files=files_to_upload if files_to_upload else None,
                headers=self._headers(),
                timeout=self.timeout
            )
            r.raise_for_status()
            result = r.json()
            return result["choices"][0]["message"]["content"], result["x_session_id"]
        
        finally:
            for _, (_, f) in files_to_upload:
                f.close()

    def chat_continue_streaming(self, model: str, session_id: str, user_message: str, agent_type: str = "auto", files: list = None) -> Iterator[str]:
        """
        Continue existing chat with streaming response
        
        Args:
            model: Model name
            session_id: Session ID to continue
            user_message: User message
            agent_type: Agent type (auto, react, plan_execute) - Note: streaming only works for simple chat
            files: Optional list of file paths to attach
        
        Yields:
            Response tokens as they're generated
        """
        messages = [{"role": "user", "content": user_message}]
        
        data = {
            "model": model,
            "messages": json.dumps(messages),
            "session_id": session_id,
            "agent_type": agent_type,
            "stream": "true"
        }
        
        files_to_upload = []
        if files:
            for file_path in files:
                f = open(file_path, "rb")
                files_to_upload.append(("files", (Path(file_path).name, f)))
        
        try:
            with httpx.stream(
                "POST",
                f"{self.base_url}/v1/chat/completions",
                data=data,
                files=files_to_upload if files_to_upload else None,
                headers=self._headers(),
                timeout=self.timeout
            ) as response:
                response.raise_for_status()
                
                for line in response.iter_lines():
                    if line.startswith("data: "):
                        data_str = line[6:]
                        
                        if data_str == "[DONE]":
                            break
                        
                        try:
                            chunk = json.loads(data_str)
                            
                            if "error" in chunk:
                                raise Exception(f"Streaming error: {chunk['error']['message']}")
                            
                            if "choices" in chunk and len(chunk["choices"]) > 0:
                                delta = chunk["choices"][0].get("delta", {})
                                if "content" in delta:
                                    yield delta["content"]
                        
                        except json.JSONDecodeError:
                            continue
        
        finally:
            for _, (_, f) in files_to_upload:
                f.close()

    def chat_sessions(self):
        r = httpx.get(f"{self.base_url}/api/chat/sessions", headers=self._headers(), timeout=10.0)
        r.raise_for_status()
        return r.json()["sessions"]

    def chat_history(self, session_id: str):
        r = httpx.get(f"{self.base_url}/api/chat/history/{session_id}", headers=self._headers(), timeout=10.0)
        r.raise_for_status()
        return r.json()["messages"]

    def tools(self):
        r = httpx.get(f"{self.base_url}/api/tools/list", headers=self._headers(), timeout=10.0)
        r.raise_for_status()
        return r.json()["tools"]

    def websearch(self, query: str, max_results: int = 5):
        """
        Perform web search and get LLM-generated answer
        
        Returns:
            Dictionary with:
            - answer: LLM-generated answer from search results
            - results: Raw search results (list of dicts with title, url, content, score)
            - sources_used: List of URLs used as sources
        """
        headers = {"Authorization": f"Bearer {self.token}", "Content-Type": "application/json"} if self.token else {"Content-Type": "application/json"}
        # Increased timeout to 1 hour (3600s) for web search + LLM answer generation
        r = httpx.post(f"{self.base_url}/api/tools/websearch", json={"query": query, "max_results": max_results}, headers=headers, timeout=3600.0)
        r.raise_for_status()
        return r.json()  # Returns full response with answer, results, and sources_used

    def answer_from_json(self, model: str, json_blob: dict, question: str):
        prompt = f"Given this JSON: {json_blob}\nAnswer: {question}"
        return self.chat_new(model, prompt)[0]

client = LLMApiClient(API_BASE_URL, timeout=3600.0)  # 1 hour timeout
print("Client ready with 3600s (1 hour) timeout for all requests")
print("✓ Now supports multipart/form-data with optional file attachments")
print("✓ Now supports streaming responses via chat_new_streaming() and chat_continue_streaming()")

# 1) Create a new account (skip if user already exists)

In [ ]:

username = "leesihun"
password = "s.hun.lee"
try:
    result = client.signup(username, password)
    print(f"Account created: {result}")
except Exception as e:
    print(f"Signup skipped (user may already exist): {e}")
    print("Continuing with existing account...")

# 2) Login

In [ ]:

login = client.login(username, password)
login

# 3) Change models (admin only) – optional

In [ ]:
# Using deepseek-r1:1.5b for all examples
client.login("admin", "administrator")
# client.change_model("gpt-oss:120b")

In [ ]:
# List models (OpenAI-compatible)
models = client.list_models()
models

MODEL = models["data"][0]["id"]

# 4) Start a new chat and get a response

In [ ]:
reply, session_id = client.chat_new(MODEL, "Hello! Give me a short haiku about autumn.")
reply, session_id

# 5) Continue an existing chat

In [ ]:

reply2, _ = client.chat_continue(MODEL, session_id, "Now do one about winter.")
reply2

# 6) See chat history

In [ ]:

client.chat_sessions(), client.chat_history(session_id)

# 7) Websearch with LLM-generated answer

In [ ]:

# # The API now generates an answer from search results using LLM
# client.login("leesihun", "s.hun.lee")
# search_query = "Tell me who is SiHun Lee"
# search_response, _ = client.chat_continue(MODEL, session_id, search_query, agent_type = 'react')

# print("=== LLM-Generated Answer ===")
# print(search_response)


In [ ]:
# # 7b) Websearch example - Sports news
# # Another example showing the LLM answer generation
# client.login("leesihun", "s.hun.lee")
# search_query = "What was the latest game of Liverpool FC and who won? The current date is 2025/12/12"
# search_response, _ = client.chat_new(MODEL, search_query, agent_type = 'react')

# print("=== LLM-Generated Answer ===")
# print(search_response)

# 8) Agentic tool usage - Let the LLM decide which tool to use

In [ ]:
math_reply, _ = client.chat_continue(MODEL, session_id, "What is 11.951/3.751?", agent_type='react')
print("Math Question Response:")
from IPython.display import display, Math, Latex
display(Latex(math_reply))
print(math_reply)
print("\n" + "="*80 + "\n")

# 9) Sequential reasoning

In [ ]:

# This triggers the ReAct agent because it requires step-by-step thinking
sequential_query = """
First, search the web to find the latest population of Tokyo.
Then, calculate what 15% of that population would be.
Finally, tell me the result.
Think hard, try to answer to best of your knowledge
"""
react_reply, _ = client.chat_continue(MODEL, session_id, sequential_query)
print("Sequential Reasoning (ReAct) Response:")

display(Latex(react_reply))
print("\n" + "="*80 + "\n")

# 10) Plan-and-Execute agent with multiple tools

In [ ]:

# This triggers Plan-and-Execute agent because it uses "and" for parallel tasks
parallel_query = """
Search for the latest news about artificial intelligence and
calculate the result of (100 * 0.15 + 25) / 2 and
Think about what god is and
What the best smart phone is and
what is 1007*1007/4524753.
"""
plan_reply, _ = client.chat_continue(MODEL,session_id,  parallel_query, agent_type="plan_execute")
print("Plan-and-Execute Response:")

display(Latex(plan_reply))
print("\n" + "="*80 + "\n")

# 11) Auto agent selection - Let the router decide

In [ ]:

# The smart router will analyze the query and pick the best agent
auto_query = "If the capital of France has a population of 2.1 million, and we need to allocate 500 euros per person for a project, what's the total budget needed? First search for the actual population, then calculate."
auto_reply, _ = client.chat_continue(MODEL, session_id, auto_query, agent_type="auto")
print("Auto Agent Selection Response:")
print(auto_reply)

# 12) Complex JSON data analysis

In [ ]:
# 12) Complex JSON data analysis (with File Upload)
import json

# Create a realistic e-commerce dataset
complex_json = {
    "company": "TechMart Inc",
    "quarter": "Q3 2025",
    "departments": [
        {
            "name": "Electronics",
            "employees": 45,
            "sales": [
                {"product": "Laptop", "units_sold": 320, "price": 1200, "revenue": 384000},
                {"product": "Smartphone", "units_sold": 856, "price": 800, "revenue": 684800},
                {"product": "Tablet", "units_sold": 142, "price": 500, "revenue": 71000}
            ]
        },
        {
            "name": "Home Appliances",
            "employees": 32,
            "sales": [
                {"product": "Refrigerator", "units_sold": 89, "price": 1500, "revenue": 133500},
                {"product": "Washing Machine", "units_sold": 124, "price": 900, "revenue": 111600},
                {"product": "Microwave", "units_sold": 267, "price": 200, "revenue": 53400}
            ]
        },
        {
            "name": "Furniture",
            "employees": 28,
            "sales": [
                {"product": "Desk", "units_sold": 178, "price": 450, "revenue": 80100},
                {"product": "Chair", "units_sold": 432, "price": 150, "revenue": 64800},
                {"product": "Bookshelf", "units_sold": 95, "price": 300, "revenue": 28500}
            ]
        }
    ]
}

# Save the data to a JSON file
json_name = './complex_json.json'
with open(json_name, 'w') as f:
    json.dump(complex_json, f, indent=2)

# Ask the LLM to analyze the attached JSON file
analysis_query = """
Analyze the attached company data JSON file and tell me:
1. Which department has the highest total revenue?
2. What is the average revenue per employee across all departments?
3. Which single product generated the most revenue?
4. Calculate the total units sold across all departments.

Please provide exact numbers and show your calculations.
"""

# Expected Answers:
# 1. Electronics: 1,139,800
# 2. 15,356.19 (1,615,300 total revenue / 105 total employees)
# 3. Smartphone: 684,800
# 4. 2,503 units total

# NEW: Upload the JSON file instead of pasting data in prompt
json_reply, _ = client.chat_continue(
    MODEL, 
    session_id,
    analysis_query,
    files=[json_name]
)

print("\n=== Complex JSON Analysis Response ===")
print(json_reply)

# Cleanup
Path(json_name).unlink()
print(f"\n✓ Cleaned up {json_name}")

## 13) Real Data Analysis - Warpage Statistics

Analyze real manufacturing warpage measurement data from uploaded JSON file.

In [ ]:
# 13) Real Data Analysis - Warpage Statistics from JSON
# Load and analyze the actual warpage analysis report data
import json
from pathlib import Path

stats_path = Path(f"data/uploads/{username}/20251013_stats.json")
with open(stats_path, 'r') as f:
    warpage_stats = json.load(f)

analysis_query = """
The given file contains 50 measurements of warpage.
Based on this warpage analysis data, please analyze and tell me:

1. Which has the highest maximum warpage value and what is it?
2. Which has the lowest minimum warpage value and what is it?
3. What is the average mean warpage across all 50 measurements?
4. Calculate the overall standard deviation range (min std to max std) across all measurements
5. Which measurement shows the most variability (highest range) and what is that range?
6. What is the average kurtosis value across all measurements?
7. Identify any files with extreme kurtosis (>47) which might indicate outliers

Please provide specific file IDs and numeric values in your analysis.
"""

warpage_reply, _ = client.chat_new(MODEL, analysis_query, files=[stats_path])
print("=== Warpage Statistics Analysis ===")
from IPython.display import display, Math, Latex
display(Latex(warpage_reply))
print("\n" + "="*80)

"""
Answer: 


1. Highest Maximum Warpage Value:
File ID: File_48
Filename: 20251013146640@B5913326505D7_ORI.txt
Maximum Value: 534.0
2. Lowest Minimum Warpage Value:
File ID: File_04
Filename: 20251013142156@B5913326505D7_ORI.txt
Minimum Value: -4200.0
3. Average Mean Warpage:
-297.84 (across all 50 measurements)
4. Overall Standard Deviation Range:
Minimum Std: 71.23
Maximum Std: 87.89
Range: 16.66
5. Most Variability (Highest Range):
File ID: File_04
Filename: 20251013142156@B5913326505D7_ORI.txt
Range: 4723.0
6. Average Kurtosis Value:
44.43 (across all measurements)
7. Files with Extreme Kurtosis (>47) - Potential Outliers: Found 7 files with extreme kurtosis:
File ID	Filename	Kurtosis
File_48	20251013146640@...	48.35
File_36	20251013145428@...	48.23
File_24	20251013144216@...	48.12
File_04	20251013142156@...	47.89
File_40	20251013145832@...	47.57
File_28	20251013144620@...	47.46
File_12	20251013143004@...	47.12
"""

# 14) Python Code Generation - Simple Calculation

In [ ]:

# Let the agent automatically generate and execute Python code
calculation_query = """
Calculate the Fibonacci sequence up to 100.
Use an efficient iterative approach and print the result as a JSON list.

Along with the results, please provide the code.
"""

python_reply, _ = client.chat_continue(MODEL,session_id,  calculation_query)
print("Python Code Generation Response:")
print(python_reply)

# 15) Python Code Generation - Data Analysis

In [ ]:

# Generate code to analyze data with pandas
data_analysis_query = """
Write Python code to:
1. Create a pandas DataFrame with 100 rows of random sales data (date, product, quantity, price)
2. Calculate total revenue per product
3. Find the top 3 products by revenue
4. Output results as JSON

Use numpy for random data generation.
"""

data_reply, _ = client.chat_continue(MODEL, session_id, data_analysis_query)
print("Data Analysis Code Response:")
print(data_reply)

# 16) Python Code Generation - Mathematical Computation

In [ ]:

# Generate code for complex mathematical calculations
math_query = """
Write Python code to:
1. Calculate the first 20 prime numbers
2. Compute their sum and average
3. Find the largest prime in the list
4. Output results as JSON with keys: primes, sum, average, largest

Show the code.
"""

math_reply, _ = client.chat_continue(MODEL, session_id, math_query, agent_type="auto")
print("Mathematical Computation Response:")
print(math_reply)

# 17) Python Code Generation - String Processing

In [ ]:

# Generate code for text analysis
text_query = """
Write Python code to analyze the following text:
"The quick brown fox jumps over the lazy dog. The dog was not amused."

Calculate:
1. Total word count
2. Unique word count
3. Most frequent word
4. Average word length
5. Output as JSON
"""

text_reply, _ = client.chat_continue(MODEL, session_id, text_query, agent_type="auto")
print("Text Processing Response:")
print(text_reply)

# 18) Python Code Generation - Excel File Analysis

In [ ]:
# 19) File Upload with Chat - CSV Analysis
import pandas as pd

print("=== Testing File Upload with Chat ===\n")

# Create test CSV file
test_data = pd.DataFrame({
    "name": ["Alice", "Bob", "Charlie", "David", "Eve"],
    "age": [30, 25, 35, 28, 32],
    "city": ["Seoul", "Busan", "Incheon", "Daegu", "Seoul"],
    "salary": [50000, 45000, 55000, 48000, 52000]
})
test_csv_path = "test_employee_data.csv"
test_data.to_csv(test_csv_path, index=False)
print(f"✓ Created test file: {test_csv_path}")
print(f"  Data shape: {test_data.shape}")

# Send chat with file attachment
query = """
Analyze the attached employee CSV file and tell me:
1. Average age and salary
2. City with most employees
3. Highest and lowest salary
4. Any interesting patterns
"""

reply, session_id = client.chat_continue(
    model=MODEL,
    session_id = session_id, 
    user_message=query,
    agent_type="auto",
    files=[test_csv_path]
)

print("\n=== AI Response ===")
print(reply)
print(f"\nSession ID: {session_id}")

# Cleanup
Path(test_csv_path).unlink()
print(f"\n✓ Cleaned up test file")

# 19) File Upload with Chat - CSV Data Analysis

Test the new file upload feature by creating and analyzing a CSV file.

In [ ]:
# 18) Python Code Generation - Excel File Analysis (with File Upload)
# Now using the new file attachment feature!

excel_path = f"data/uploads/{username}/폴드긍정.xlsx"

excel_analysis_query = """
Analyze the attached Excel file and:
1. What's the most widely appreciated feature?
2. What's the most widely used phrase?
3. What's the least used phrase?

Make sure to handle Korean text encoding properly.
"""

# NEW: Upload the file with the chat request
excel_reply, _ = client.chat_continue(
    MODEL, 
    session_id, 
    excel_analysis_query, 
    agent_type="auto",
    files=[excel_path]  # Attach the file!
)

print("Excel File Analysis Response:")
print(excel_reply)

# 20. 끝말잇기

In [ ]:
query = '끝말 잇기하자 나부터 시작할게: 이시훈'

reply, _ = client.chat_continue(MODEL, session_id, query)
print(reply)

for i in range(10):
    reply, _ = client.chat_continue(MODEL, session_id, query)
    print(reply)

In [ ]:
import time

question = "Explain quantum computing in 3 sentences"

# Test non-streaming
print("=== Non-Streaming (traditional) ===")
start = time.time()
response, _ = client.chat_new(MODEL, question)
elapsed_non_stream = time.time() - start
print(f"Response: {response}")
print(f"⏱ Total time: {elapsed_non_stream:.2f}s")

print("\n" + "="*80 + "\n")

# Test streaming
print("=== Streaming (real-time) ===")
start = time.time()
first_token_time = None
print("Response: ", end="", flush=True)

for i, token in enumerate(client.chat_new_streaming(MODEL, question)):
    if not token.startswith("SESSION_ID:"):
        if first_token_time is None:
            first_token_time = time.time() - start
        print(token, end="", flush=True)

elapsed_stream = time.time() - start

print(f"\n\n⏱ Time to first token: {first_token_time:.2f}s")
print(f"⏱ Total time: {elapsed_stream:.2f}s")
print(f"\n✓ Streaming advantage: User sees response {first_token_time:.2f}s faster!")

# 23) Streaming vs Non-Streaming Comparison

Compare streaming and non-streaming response times.

In [ ]:
# Example 2: Continue streaming conversation
if session_id:
    print("=== Continuing Conversation (Streaming) ===\n")
    print("Question: What happened next in the story?\n")
    print("Response: ", end="", flush=True)
    
    for token in client.chat_continue_streaming(MODEL, session_id, "What happened next in the story?"):
        print(token, end="", flush=True)
    
    print("\n\n✓ Stream complete!")
else:
    print("⚠ No session_id from previous example - run the previous cell first")

# 22) Streaming Response - Continue Conversation

Continue the conversation from the previous streaming example.

In [ ]:
# Example 1: Basic streaming with visual feedback
print("=== Streaming Chat Example ===\n")
print("Question: Tell me a short story about a robot learning to cook\n")
print("Response: ", end="", flush=True)

session_id = None
full_response = []

for token in client.chat_new_streaming(MODEL, "Tell me a short story about a robot learning to cook"):
    # Check if this is the session_id
    if token.startswith("SESSION_ID:"):
        session_id = token.replace("SESSION_ID:", "")
    else:
        # It's a content token
        print(token, end="", flush=True)
        full_response.append(token)

print(f"\n\n✓ Stream complete!")
print(f"✓ Session ID: {session_id}")
print(f"✓ Total characters received: {len(''.join(full_response))}")

# 21) Streaming Response Example - Simple Chat

Test the new streaming feature for real-time token-by-token responses.